In [1]:
%useLatestDescriptors
%use dataframe
%use ktor-client

@file:DependsOn("org.jetbrains.kotlinx:kotlinx-io-core:0.8.0")
@file:DependsOn("org.apache.commons:commons-compress:1.28.0")

In [17]:
import kotlinx.io.asInputStream
import kotlinx.io.buffered
import kotlinx.io.files.Path
import kotlinx.io.files.SystemFileSystem
import org.apache.commons.compress.archivers.tar.TarArchiveEntry
import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream
import org.apache.poi.util.IOUtils

private val remoteDataUrl = "https://github.com/ageron/handson-ml/raw/master/datasets"
private val housingUrl = "$remoteDataUrl/housing/housing.tgz"

private val localDataDir = "../datasets"
private val housingTgzFile = "housing.tgz"
private val housingCsvFile = "housing.csv"

fun getHousingTgz() : Path {
    val housingTgzPath = Path("$localDataDir/$housingTgzFile")
    val data = http.get(housingUrl).readBytes()
    SystemFileSystem.createDirectories(Path(localDataDir))
    SystemFileSystem.sink(housingTgzPath).buffered().use {
        it.write(data)
    }
    return housingTgzPath
}

fun extractHousingCsv(path: Path) : Path {
    val housingCsvPath = Path("$localDataDir/$housingCsvFile")
    SystemFileSystem.source(path).buffered().use { source ->
        GzipCompressorInputStream(source.asInputStream()).use { gins ->
            TarArchiveInputStream(gins).use { tais ->
                var entry = tais.nextEntry
                while (entry != null) {
                    if (!entry.isDirectory && entry.name.endsWith(".csv")) {
                        SystemFileSystem.sink(housingCsvPath).buffered().use { sink ->
                            sink.write(tais.readBytes())
                        }
                    }
                    entry = tais.nextEntry
                }
            }
        }
    }
    return housingCsvPath
}

In [18]:
import kotlinx.io.write

val housingTgz = getHousingTgz()
val housingCsv = extractHousingCsv(housingTgz)
val dataFrame = DataFrame.readCsv(File(housingCsv.toString()))


housing.csv

longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
"-122,230000","37,880000","41,000000","880,000000","129,000000","322,000000","126,000000","8,325200","452600,000000",NEAR BAY
"-122,220000","37,860000","21,000000","7099,000000","1106,000000","2401,000000","1138,000000","8,301400","358500,000000",NEAR BAY
"-122,240000","37,850000","52,000000","1467,000000","190,000000","496,000000","177,000000","7,257400","352100,000000",NEAR BAY
"-122,250000","37,850000","52,000000","1274,000000","235,000000","558,000000","219,000000","5,643100","341300,000000",NEAR BAY
"-122,250000","37,850000","52,000000","1627,000000","280,000000","565,000000","259,000000","3,846200","342200,000000",NEAR BAY
"-122,250000","37,850000","52,000000","919,000000","213,000000","413,000000","193,000000","4,036800","269700,000000",NEAR BAY
"-122,250000","37,840000","52,000000","2535,000000","489,000000","1094,000000","514,000000","3,659100","299200,000000",NEAR BAY
"-122,250000","37,840000","52,000000","3104,000000","687,000000","1157,000000","647,000000","3,120000","241400,000000",NEAR BAY
"-122,260000","37,840000","42,000000","2555,000000","665,000000","1206,000000","595,000000","2,080400","226700,000000",NEAR BAY
"-122,250000","37,840000","52,000000","3549,000000","707,000000","1551,000000","714,000000","3,691200","261100,000000",NEAR BAY
